# Validate & Create Test Scenarios

**Source:** https://confluence.hl7.org/pages/viewpage.action?pageId=204276132

**Run this notebook in the cloud:**

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/beda-software/au-core-aidbox/main?labpath=client%2FValidate+%26+Create+Test+Scenarios.ipynb)

In [ ]:
from fhirpy import SyncFHIRClient
from pprint import pprint
from fhirpy.base.exceptions import OperationOutcome
import uuid
import copy
import json

In [ ]:
SPARKED_BASE_URL = "https://hl7auconnectathon.salessbx.smiledigitalhealth.com/fhir-request"
AIDBOX_BASE_URL = "https://au-core.beda.software/fhir/"

In [ ]:
client = SyncFHIRClient(AIDBOX_BASE_URL)

In [ ]:
FHIR_SCHEMA_VALIDATOR = "https://fhir-validator.aidbox.app"
validator = SyncFHIRClient("https://fhir-validator.aidbox.app")

def validate_resource(resource):
    return pprint(validator.execute("rpc", data={
        "method":"fhirpathweb/validate-resource",
        "params":{
            "resource": json.dumps(resource),
            "fhir-version":"4.0.1",
            "format":"json",
            "disabled-features":{}}})["result"])

def remove_attributes(resource_data, attributes_list):
    d = copy.deepcopy(resource_data)
    for attribute in attributes_list:
        d.pop(attribute, None)

    return d

## 1. Patient
### 1.1 $validate Patient
**Action:**  Validate the content of a FHIR formatted Patient resource

In [ ]:
valid_patient_data = {
 "meta": {
  "profile": [
   "http://hl7.org.au/fhir/core/StructureDefinition/au-core-patient"
  ],
 },
 "name": [
  {
   "given": [
    "Valid"
   ],
   "family": "Patient",
  }
 ],
 "birthDate": "1999-12-19",
 "resourceType": "Patient",
 "id": str(uuid.uuid4()),
 "identifier": [
  {
   "type": {
    "text": "Medicare Number",
    "coding": [
     {
      "code": "MC",
      "system": "http://terminology.hl7.org/CodeSystem/v2-0203",
      "display": "Patient's Medicare Number"
     }
    ]
   },
   "value": "1234567892",
   "period": {
    "end": "2024-05"
   },
   "system": "http://ns.electronichealth.net.au/id/medicare-number"
  }
 ],
 "gender": "female",
}

invalid_patient_data = remove_attributes(valid_patient_data, ["identifier", "birthDate"])

In [ ]:
validate_resource(valid_patient_data)

### 1.2 create Patient
**Action:** FHIR client creates a new patient and save to FHIR server.

In [ ]:
patient = client.resource("Patient", **valid_patient_data)
patient.create()

In [ ]:
example_patient = client.reference('Patient', valid_patient_data["id"]).to_resource()
pprint(example_patient.serialize())

## 2. Condition
### 2.1 $validate Condition
**Action:**  Validate the content of a FHIR formatted Condition resource

In [ ]:
valid_condition_data = {
 "onset": {
  "dateTime": "2017-03-30"
 },
 "category": [
  {
   "coding": [
    {
     "code": "encounter-diagnosis",
     "system": "http://terminology.hl7.org/CodeSystem/condition-category",
     "display": "Encounter Diagnosis"
    }
   ]
  }
 ],
 "clinicalStatus": {
  "coding": [
   {
    "code": "active",
    "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
    "display": "Active"
   }
  ]
 },
 "meta": {
  "profile": [
   "http://hl7.org.au/fhir/core/StructureDefinition/au-core-condition"
  ],
 },
 "encounter": {
  "id": "nailwound",
  "resourceType": "Encounter"
 },
 "resourceType": "Condition",
 "id": str(uuid.uuid4()),
 "recorder": {
  "id": "bobrester-bob-gp",
  "resourceType": "PractitionerRole"
 },
 "code": {
  "coding": [
   {
    "code": "283680004",
    "system": "http://snomed.info/sct",
    "display": "Nail wound of sole of foot"
   }
  ]
 },
 "bodySite": [
  {
   "coding": [
    {
     "code": "57999000",
     "system": "http://snomed.info/sct",
     "display": "Structure of sole of foot"
    }
   ]
  }
 ],
 "verificationStatus": {
  "coding": [
   {
    "code": "confirmed",
    "system": "http://terminology.hl7.org/CodeSystem/condition-ver-status",
    "display": "Confirmed"
   }
  ]
 },
 "subject": {
  "id": "wang-li",
  "resourceType": "Patient"
 },
}

invalid_condition_data = remove_attributes(valid_condition_data, ["category", "code"])

In [ ]:
validate_resource(valid_condition_data)

### 2.2 create Condition
**Action:** FHIR client creates a new Condition instance for a patient and save to FHIR server.

In [ ]:
condition = client.resource("Condition", **valid_condition_data)
condition.create()

In [ ]:
example_condition = client.reference('Condition', valid_condition_data["id"]).to_resource()
pprint(example_condition.serialize())

## 3. AllergyIntolerance
### 3.1 $validate AllergyIntolerance
**Action:**  Validate the content of a FHIR formatted resource

In [ ]:
valid_allergy_data = {
 "patient": {
  "id": "dan-harry",
  "resourceType": "Patient"
 },
 "onset": {
  "dateTime": "2023-02-12"
 },
 "category": [
  "food"
 ],
 "clinicalStatus": {
  "text": "Active",
  "coding": [
   {
    "code": "active",
    "system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-clinical",
    "display": "Active"
   }
  ]
 },
 "meta": {
  "profile": [
   "http://hl7.org.au/fhir/core/StructureDefinition/au-core-allergyintolerance"
  ],
 },
 "type": "allergy",
 "resourceType": "AllergyIntolerance",
 "asserter": {
  "id": "bobrester-bob-gp",
  "resourceType": "PractitionerRole"
 },
 "recordedDate": "2023-04-24",
 "id": str(uuid.uuid4()),
 "recorder": {
  "id": "bobrester-bob-gp",
  "resourceType": "PractitionerRole"
 },
 "code": {
  "text": "Egg",
  "coding": [
   {
    "code": "102263004",
    "system": "http://snomed.info/sct",
    "display": "Eggs (edible)"
   }
  ]
 },
 "verificationStatus": {
  "text": "Confirmed",
  "coding": [
   {
    "code": "confirmed",
    "system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-verification",
    "display": "Confirmed"
   }
  ]
 },
 "reaction": [
  {
   "substance": {
    "text": "Egg",
    "coding": [
     {
      "code": "102263004",
      "system": "http://snomed.info/sct",
      "display": "Eggs (edible)"
     }
    ]
   },
   "manifestation": [
    {
     "text": "Stomach cramps",
     "coding": [
      {
       "code": "51197009",
       "system": "http://snomed.info/sct",
       "display": "Stomach cramps"
      }
     ]
    },
    {
     "text": "Nausea and vomiting",
     "coding": [
      {
       "code": "16932000",
       "system": "http://snomed.info/sct",
       "display": "Nausea and vomiting"
      }
     ]
    }
   ]
  }
 ],
}

invalid_allergy_data = remove_attributes(valid_allergy_data, ["code"])

In [ ]:
validate_resource(valid_allergy_data)

### 3.2 create AllergyIntolerance
**Action:** FHIR client creates a new AllergyIntolerance instance for a patient and save to FHIR server.

In [ ]:
allergy = client.resource("AllergyIntolerance", **valid_allergy_data)
allergy.create()

In [ ]:
example_allergy = client.reference('AllergyIntolerance', valid_allergy_data["id"]).to_resource()
pprint(example_allergy.serialize())